# RAG with Azure AI search

In [1]:
# Import required libraries
import os
import json
from dotenv import load_dotenv

from langchain_openai import AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Configure environment variables
load_dotenv()

True

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SearchField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)

In [3]:
from openai import AzureOpenAI
# Configure environment variables
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")
OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4_DEPLOYMENT_NAME")
OPENAI_GPT4V_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4V_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_DALLE_DEPLOYMENT_NAME = os.getenv("OPENAI_DALLE_DEPLOYMENT_NAME")

OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Configure OpenAI API
client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)
credential = AzureKeyCredential(key)

In [4]:
# Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:
embeddingmodel = AzureOpenAIEmbeddings(
    deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    chunk_size = 1)

In [5]:
# Generate Document Embeddings using OpenAI Ada Model
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    embeddings = embeddingmodel.embed_query(text)
    return embeddings

## Perform a vector similarity search

In [6]:
# Pure Vector Search
query = "semantic kernel?"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query),
    top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.8773118
Content: Tell us about y our PDF experience.What is Semantic Kernel?
Article •07/11/2023
Semantic K ernel is an open-source SDK that lets you easily combine AI services like
OpenAI , Azure OpenAI , and Hugging F ace  with conventional programming
languages like C# and Python. By doing so, you can create AI apps that combine the
best of both worlds.
During K evin Scott's talk The era of the AI Copilot , he showed how Microsoft powers its
Copilot system  with a stack of AI models and plugins. At the center of this stack is an AI
orchestration layer that allows us to combine AI models and plugins together to create
brand new experiences for users.
Semantic Kernel is at the center of the copilot
stack
Title: Semantic Kernel
Score: 0.8760598
Content: Responsible AI and Semantic Kernel
Article •05/23/2023
An AI system includes not only the technology, but also the people who will use it, the
people who will be affected by it, and the environment in whi

In [7]:
query = "semantic kernel planner and kernel"

search_client = SearchClient(
    service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.87037444
Content: To simplify the creation of AI apps, open source projects like LangChain  have
emerged. Semantic K ernel is Microsoft's contribution to this space and is designed to
support enterprise app developers who want to integrate AI into their existing apps.
By using multiple AI models, plugins, and memory all together within Semantic K ernel,
you can create sophisticated pipelines that allow AI to automate complex tasks for users.
For example, with Semantic K ernel, you could create a pipeline that helps a user send an
email to their marketing team. With memory , you could retrieve information about the
project and then use planner  to autogenerate the remaining steps using available
plugins (e.g., ground the user's ask with Microsoft Graph data, generate a response with
GPT-4, and send the email). Finally, you can display a success message back to your user
in your app using a custom plugin.
Step Component Descr iption
1 Ask It starts with a 

In [8]:
# Pure Vector Search multi-lingual
query = "Planificador semántico del kernel y kernel"

search_client = SearchClient(
    service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.83616686
Content: Responsible AI and Semantic Kernel
Article •05/23/2023
An AI system includes not only the technology, but also the people who will use it, the
people who will be affected by it, and the environment in which it is deployed. Creating
a system that is fit for its intended purpose requires an understanding of how the
technology works, what its capabilities and limitations are, and how to achieve the best
performance. Microsoft’s T ransparency Notes are intended to help you understand how
our AI technology works, the choices system owners can make that influence system
performance and behavior, and the importance of thinking about the whole system,
including the technology, the people, and the environment. Y ou can use T ransparency
Notes when developing or deploying your own system, or share them with the people
who will use or be affected by your system.
Microsoft’s T ransparency Notes are part of a broader effort at Microsoft to put our A

## Perform Cross-Field Vector Search with a filter

In [9]:
# Pure Vector Search with Filter
query = "programming languages supported by semantic kernel"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=None,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="titleVector, contentVector",
    filter="title eq 'Semantic Kernel'",
    select=["title", "content"] #searching on two fields title and content
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")

Title: Semantic Kernel
Score: 0.01666666753590107
Content: Tell us about y our PDF experience.What is Semantic Kernel?
Article •07/11/2023
Semantic K ernel is an open-source SDK that lets you easily combine AI services like
OpenAI , Azure OpenAI , and Hugging F ace  with conventional programming
languages like C# and Python. By doing so, you can create AI apps that combine the
best of both worlds.
During K evin Scott's talk The era of the AI Copilot , he showed how Microsoft powers its
Copilot system  with a stack of AI models and plugins. At the center of this stack is an AI
orchestration layer that allows us to combine AI models and plugins together to create
brand new experiences for users.
Semantic Kernel is at the center of the copilot
stack
Title: Semantic Kernel
Score: 0.01666666753590107
Content: Supported Semantic Kernel languages
Article •07/18/2023
Semantic K ernel plans on providing support to the following languages:
While the overall architecture of the kernel is consiste

## Perform an Hybrid Search

In [10]:
# Hybrid Search
query = "semantic kernel planner and kernel"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)


results = search_client.search(
    search_text=query,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    filter="title eq 'Semantic Kernel'",
    select=["title", "content",],
    top=3
)

print(type(results))

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}\n")

<class 'azure.search.documents._paging.SearchItemPaged'>
Title: Semantic Kernel
Score: 0.03306011110544205
Content: To simplify the creation of AI apps, open source projects like LangChain  have
emerged. Semantic K ernel is Microsoft's contribution to this space and is designed to
support enterprise app developers who want to integrate AI into their existing apps.
By using multiple AI models, plugins, and memory all together within Semantic K ernel,
you can create sophisticated pipelines that allow AI to automate complex tasks for users.
For example, with Semantic K ernel, you could create a pipeline that helps a user send an
email to their marketing team. With memory , you could retrieve information about the
project and then use planner  to autogenerate the remaining steps using available
plugins (e.g., ground the user's ask with Microsoft Graph data, generate a response with
GPT-4, and send the email). Finally, you can display a success message back to your user
in your app using a 

## Perform a Semantic Hybrid Search

In [11]:
# Semantic Hybrid Search
query = "semantic kernel planner and kernel"

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index-2", credential=credential)

results = search_client.search(
    search_text=query,
    vector=generate_embeddings(query), top_k=3,
    vector_fields="contentVector",
    select=["title", "content"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='sk-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Content: {result['content']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer: Semantic Kernel.<em> any other operation that you can do in code that is ill-suited for</em> LLMs<em> (e.g., performing calculations).</em> Instead of providing a separate configuration file with semantic descriptions,<em> planner is able to use annotations in the code to understand how the function behaves.</em>
Semantic Answer Score: 0.71630859375

Title: Semantic Kernel
Content: To instantiate planner, all you need to do is pass it a kernel object. Planner will then
automatically discover all of the plugins registered in the kernel and use them to create
plans. The following code initializes both a kernel and a SequentialPlanner. At the end
of this article we'll review the other types of Planners that are available in Semantic
Kernel.
C#      return (
          Convert.ToDouble(context[ "input"], 
CultureInfo.InvariantCulture) -
          Convert.ToDouble(context[ "number2" ], 
CultureInfo.InvariantCulture)
      ).ToString(CultureInfo.InvariantCulture);
  }
  [SKFu